In [ ]:
#BUILDING DATASETS

def make_arxiv_dataset():
    start = time.time()
    sample_indices = np.random.randint(low = 0, high = len(metadata), size = 10)
    sampled_articles = ["" for _ in range(sample_size)]
    sampled_ids = ["" for _ in range(sample_size)]
    for ii, i in enumerate(sample_indices):
        article_id = metadata["id"][i]
        sampled_ids[ii] = article_id
        b = req_article(article_id)
        sampled_articles[ii] = b.decode()
    end = time.time()

    save_article_list(sampled_articles, "all_articles.txt")
    end = time.time()
    print(end-start)
    
def make_wiki_dataset():
    #wiki sentences into lists
    with open(NORMAL_WIKI_PATH, "rt") as f:
        normal = f.readlines()
    with open(SIMPLE_WIKI_PATH, "rt") as f:
        simple = f.readlines()
    #collect titles and articles
    normal_ds = {}
    curr_sentences = []
    curr_title = normal[0].split("\t")[0] #initialize with first title
    for a in normal:
        split = a.split("\t")
        title, para, sentence = split[0], split[1], split[2][:-1] #drop new line character

        if title != curr_title:
            art = " ".join(curr_sentences)
            normal_ds[curr_title] = art
            curr_title = title
            curr_sentences = []        
        curr_sentences.append(sentence)

    art = " ".join(curr_sentences)
    normal_ds[curr_title] = art



    #simple wiki
    simple_ds = {}
    curr_sentences = []
    curr_title = simple[0].split("\t")[0] #initialize with first title
    for a in simple:
        split = a.split("\t")
        title, para, sentence = split[0], split[1], split[2][:-1] #drop new line character

        if title != curr_title:
            art = " ".join(curr_sentences)
            simple_ds[curr_title] = art
            curr_title = title
            curr_sentences = []
        curr_sentences.append(sentence)

    art = " ".join(curr_sentences)
    simple_ds[curr_title] = art

    titles = []
    s_articles = []
    n_articles = []
    i = 0
    for k in normal_ds.keys():
        try:
            s_articles.append(simple_ds[k])
            n_articles.append(normal_ds[k])
            titles.append(k)
        except: 
            print("error thrown, probably a non-common key")
    ds = {"titles": titles, "simple_articles": s_articles, "normal_articles": n_articles}
    hf_ds = Dataset.from_dict(ds)
    hf_ds.save_to_disk("../res/datasets/wiki_aligned/aligned_wiki_ds")
    